<a href="https://colab.research.google.com/github/s-duuu/YOLO_Project/blob/main/YOLOv5s_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 런타임 -> 런타임 유형 변경 -> 하드웨어 가속기 GPU로 설정

In [ ]:
# 구글 드라이브에 마운트
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 내 구글 드라이브로 이동
# %cd "/content/drive/MyDrive"

# YOLOv5 github repository clone
!git clone https://github.com/ultralytics/yolov5.git

# 필요한 모듈 설치
!pip install -U -r yolov5/requirements.txt

In [ ]:
import torch

# 파이토치 버전 확인, cuda device properties 확인
print('torch %s %s' % (torch.__version__, torch.cuda.geet_device_properties(0) if torch.cuda.is_available() else 'CPU'))

# 여기서 CPU라고 출력된다면 런타임 -> 런타임 유형 변경 -> 하드웨어 가속기 GPU로 설정되어있는지 확인

In [ ]:
!mkdir datasets
%cd /content/datasets/

In [ ]:
# Roboflow data (라벨링까지 다 되어있는 오픈소스 데이터셋) download
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="GgExW6M2mEEMfnV6KTpL")
project = rf.workspace("gaurigodghase-gmail-com").project("vehicles-openimages-svzce")
dataset = project.version(1).download("yolov5")

In [ ]:
# data.yaml 파일 수정
from glob import glob

# 이미지 경로 list로 넣기
train_img_list = glob('./train_bong/images/*.jpg') + glob('./train_bong/images/*.jpeg')
valid_img_list = glob('./valid_bong/images/*.jpg') + glob('./valid_bong/images/*.jpeg')


# txt 파일에 write
with open('./train.txt', 'w') as f:
	f.write('\n'.join(train_img_list) + '\n')
    
with open('./valid.txt', 'w') as f:
	f.write('\n'.join(valid_img_list) + '\n')


**data.yaml 파일 수정**

2가지 파라미터를 수정해줘야함

1st) train: ________
빈칸에 train.txt 파일 경로를 입력해줘야함

2nd) val: ________
빈칸에 valid.txt 파일 경로를 입력해줘야함

파일 경로는 왼쪽 directory 아이콘에서 우클릭하면 경로 복사 가능

**YOLOv5s 학습시키기**

1. 기본 source file: train.py
2. Parameters : img, batch, epochs, data, cfg, weights, name, nosave, cache

**Parameter description**
- img: 입력 이미지 크기
- batch: 배치 크기
- epochs: 학습 epoch 수 (참고: 3000개 이상이 일반적으로 사용됨)
- data: data.yaml 파일 경로
- cfg: 모델 구성 지정
- weights: 가중치에 대한 사용자 정의 경로를 지정 -> **yolov5 공식 github에서 다운로드!!!** -> https://github.com/ultralytics/yolov5/releases
- name: 모델이 저장될 폴더 이름
- nosave: 최종 체크포인트만 저장
- cache: 더 빠른 학습을 위해 이미지를 캐시

In [ ]:
# Executing train.py (파라미터 변경해볼것)
!python /content/yolov5/train.py --img 320 --batch 16 --epochs 30 --data /content/datasets/data.yaml --weights yolov5s.pt --name result --cfg /content/yolov5/models/yolov5s.yaml

In [ ]:
# 학습 결과 다운로드
!zip -r train_result.zip /content/yolov5/runs/train/exp

In [ ]:
# Executing val.py (train.py 실행을 통해 나온 best.pt를 weight로 지정)
!python val.py --data "/content/datasets/data.yaml" -- weights "/content/yolov5/runs/train/exp/weights/best.pt"

In [ ]:
# 검증 결과 다운로드
!zip -r val_result.zip /content/yolov5/runs/val

**학습한 모델로 예측하기**

detect.py 파일의 **--soruce** parameter
- 0 : webcam
- img.jpg : image
- vid.mp4 : video
- path/ : directory
- path/*.jpg : glob

In [ ]:
# Executing detect.py (--source 파라미터 변경해볼것, 현재 예시코드에서는 roboflow dataset으로 설정해놓음)
!python detect.py --weights "/content/yolov5/runs/train/exp/weights/best.pt" -- source "/content/datasets/test/images"

In [ ]:
# 예측 테스트 결과 다운로드
!zip -r test_result.zip /content/yolov5/runs/detect/exp

# 폴더를 열어보면 class, bounding box가 표시된 detection 결과 이미지가 저장되어 있음